In [17]:
%pip install pypdf langchain pytest boto3 chromadb


   ---------------------------------------- 0.0/119.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/119.4 MB 1.7 MB/s eta 0:01:13
   ---------------------------------------- 1.0/119.4 MB 16.7 MB/s eta 0:00:08
   ---------------------------------------- 1.1/119.4 MB 8.9 MB/s eta 0:00:14
    --------------------------------------- 2.1/119.4 MB 14.7 MB/s eta 0:00:08
   - -------------------------------------- 3.1/119.4 MB 13.9 MB/s eta 0:00:09
   - -------------------------------------- 3.1/119.4 MB 14.3 MB/s eta 0:00:09
   - -------------------------------------- 4.1/119.4 MB 13.1 MB/s eta 0:00:09
   - -------------------------------------- 4.2/119.4 MB 13.4 MB/s eta 0:00:09
   - -------------------------------------- 4.8/119.4 MB 11.8 MB/s eta 0:00:10
   - -------------------------------------- 5.2/119.4 MB 12.4 MB/s eta 0:00:10
   -- ------------------------------------- 6.2/119.4 MB 12.4 MB/s eta 0:00:10
   -- ------------------------------------- 6.3/119.4 MB 12.6 

In [1]:
DATA_PATH = "data/pdfs"
CHROMA_PATH = "chroma"

In [2]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_chroma import Chroma
from get_embedding_func import get_embedding_function
import shutil

def load_documents():
    loader = PyPDFDirectoryLoader(DATA_PATH)
    return loader.load()

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)


def add_to_chroma(chunks: list[Document]):
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=get_embedding_function(),
    )
    chunks_with_ids = calculate_chunk_ids(chunks)
    
    existing_items= db.get(include=[])
    existings_ids = set(existing_items["ids"])
    print(f'Existing items: {len(existings_ids)}')
    
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existings_ids:
            new_chunks.append(chunk)
    
    if new_chunks:
        new_chunk_ids= [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("No new chunks to add")
        
def calculate_chunk_ids(chunks: list[Document]):
    last_page_id = None
    current_chunk_index = 0
    
    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"
        
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0
        
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id
        
        chunk.metadata["id"] = chunk_id
    return chunks



def clear_database():
    shutil.rmtree(CHROMA_PATH)

In [7]:
# build database from documents
documents = load_documents()
chunks = split_documents(documents)
add_to_chroma(chunks)


Existing items: 0


AttributeError: 'Chroma' object has no attribute 'persist'

In [6]:
clear_database()